In [1]:
!pip install transforms3d --quiet

In [29]:
import os
import cv2 
import matplotlib.pyplot as plt
import numpy as np
from scipy.spatial.transform import Rotation
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
import transforms3d.quaternions as quat
from scipy import signal
import scipy.fft as sfft

In [92]:
focal_length_x = 5212.5371
focal_length_y = 6255.0444
principal_point_x = 640.
principal_point_y = 512.
# camera intrinsics
K = np.array([[focal_length_x, 0, principal_point_x],
                          [0, focal_length_y,principal_point_y],
                          [0, 0, 1]])

root = '/home/lausena/developer/repos/spacecraft-pose-pose-estimation-runtime/'
reference_path = os.path.join(root, 'data/images/0a998b28bd/000.png')
target_path = os.path.join(root, 'data/images/0a998b28bd/001.png')

reference_image = cv2.imread(reference_path, cv2.COLOR_BGR2GRAY)
target_image = cv2.imread(target_path, cv2.COLOR_BGR2GRAY)

expected_random_quat = np.array([0,0,0,0])
expected_random_trans = np.array([0,0,0])
expected_translation = np.array([2.21703339,23.38332176,12.38890553])
expected_quaternion = np.array([0.99002814,-0.0722533,0.07088085,-0.09797749])

#### Detect Keypoints and extract descriptors 

**Key points** are feature points--disintct locations in an image that represent significant information about the local structure or appearce.

**Descriptors** care compact representations of the local image information surrounding keypoints. They encode the appearance or texture of the neighborhood around each keypoint.

In [105]:
def find_homography(reference_image, target_image, model):
    keypoints_reference, descriptors_reference = model.detectAndCompute(reference_image, None)
    keypoints_target, descriptors_target = model.detectAndCompute(target_image, None)

    bf = cv2.BFMatcher()
    matches = bf.knnMatch(descriptors_reference, descriptors_target, k=2)
    
    print(len(matches))
    best_matches = []
    for m, n in matches:
        if m.distance < 0.75 * n.distance:
            best_matches.append(m)

    print(len(best_matches))
    src_pts = np.float32([keypoints_reference[m.queryIdx].pt for m in best_matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([keypoints_target[m.trainIdx].pt for m in best_matches]).reshape(-1, 1, 2)

    # get the homography matrix H
    H, _ = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)

    return H

In [106]:
def decompose_homography(H, K):
    _, rotation_matrix, translation_vector, _ = cv2.decomposeHomographyMat(H, K)
    return rotation_matrix[0], translation_vector[0]

In [107]:
H = find_homography(reference_image, target_image, cv2.SIFT_create())

206
41


In [108]:
rotation_matrix, translation_vector = decompose_homography(H, K)

In [109]:
print(f'Rotation Matrix: {rotation_matrix}')
print(f'Translation Vector: {translation_vector}')

Rotation Matrix: [[-0.91878396  0.01865646 -0.39431963]
 [ 0.00782778  0.99954724  0.02905249]
 [ 0.39468311  0.02360631 -0.91851401]]
Translation Vector: [[ 1.72177602]
 [-0.12129229]
 [ 8.86798939]]


In [111]:
quaternion = Rotation.from_matrix(rotation_matrix).as_quat()
print(quaternion)
print(expected_quaternion)
print(f'Random Quat: {mean_squared_error(quaternion, expected_random_quat)}')
print(f'Actual Qaut: {mean_squared_error(quaternion, expected_quaternion)}')

[ 0.00676037  0.97939333  0.01344169 -0.20140089]
[ 0.99002814 -0.0722533   0.07088085 -0.09797749]
Random Quat: 0.24999999999999997
Actual Qaut: 0.5216929520405501
